In [1]:
#!pip install optuna

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os, optuna
import warnings
from PIL import Image
from tensorflow import keras

from google.colab import drive
drive.mount("/content/drive")

K = keras.backend
# Changing default dir
# os.chdir("/Users/pedroteche/Documents/GitHub/maize-crop-diagnose/")
# optuna.logging.set_verbosity(optuna.logging.WARNING)

In [3]:
!mkdir 'data'
!cp -r 'drive/MyDrive/maize-crop-diagnose' 'data'

In [4]:
device_name = tf.test.gpu_device_name()
if device_name != "/device:GPU:0":
    raise SystemError("GPU device not found")
print("Found GPU at: {}".format(device_name))

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB

Found GPU at: /device:GPU:0


In [5]:
class OneCycleScheduler(tf.keras.callbacks.Callback):
    def __init__(
        self,
        iterations,
        max_lr=1e-3,
        start_lr=None,
        start_mom=0.95,
        min_mom=0.85,
        last_iterations=None,
        last_lr=None,
    ):
        self.iterations = iterations
        self.max_lr = max_lr
        self.start_lr = start_lr or max_lr / 10
        self.start_mom = start_mom
        self.min_mom = min_mom
        self.last_iterations = last_iterations or iterations // 10 + 1
        self.half_iteration = (iterations - self.last_iterations) // 2
        self.last_lr = last_lr or self.start_lr / 1000
        self.iteration = 0

    def _interpolate(self, iter1, iter2, lr1, lr2):
        return (lr2 - lr1) * (self.iteration - iter1) / (iter2 - iter1) + lr1

    def on_batch_begin(self, batch, logs):
        if self.iteration < self.half_iteration:
            lr = self._interpolate(0, self.half_iteration, self.start_lr, self.max_lr)
            mom = self._interpolate(
                0, self.half_iteration, self.start_mom, self.min_mom
            )
        elif self.iteration < 2 * self.half_iteration:
            lr = self._interpolate(
                self.half_iteration, 2 * self.half_iteration, self.max_lr, self.start_lr
            )
            mom = self._interpolate(
                self.half_iteration,
                2 * self.half_iteration,
                self.min_mom,
                self.start_mom,
            )
        else:
            lr = self._interpolate(
                2 * self.half_iteration, self.iterations, self.start_lr, self.last_lr
            )
            mom = self.start_mom
        self.iteration += 1
        K.set_value(self.model.optimizer.learning_rate, lr)
        K.set_value(self.model.optimizer.momentum, mom)


class OneCycleSchedulerNoMom(tf.keras.callbacks.Callback):
    def __init__(
        self,
        iterations,
        max_lr=1e-3,
        start_lr=None,
        last_iterations=None,
        last_lr=None,
    ):
        self.iterations = iterations
        self.max_lr = max_lr
        self.start_lr = start_lr or max_lr / 10
        self.last_iterations = last_iterations or iterations // 10 + 1
        self.half_iteration = (iterations - self.last_iterations) // 2
        self.last_lr = last_lr or self.start_lr / 1000
        self.iteration = 0

    def _interpolate(self, iter1, iter2, lr1, lr2):
        return (lr2 - lr1) * (self.iteration - iter1) / (iter2 - iter1) + lr1

    def on_batch_begin(self, batch, logs):
        if self.iteration < self.half_iteration:
            lr = self._interpolate(0, self.half_iteration, self.start_lr, self.max_lr)
        elif self.iteration < 2 * self.half_iteration:
            lr = self._interpolate(
                self.half_iteration, 2 * self.half_iteration, self.max_lr, self.start_lr
            )
        else:
            lr = self._interpolate(
                2 * self.half_iteration, self.iterations, self.start_lr, self.last_lr
            )
        self.iteration += 1
        K.set_value(self.model.optimizer.learning_rate, lr)


def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

In [6]:
IMG_HEIGHT = 64 * 2
IMG_WIDTH = 48 * 2
BATCH_SIZE = 32
EPOCH = 50
TRAIN_DATA_DIR = "data/maize-crop-diagnose/data/train"
TEST_DATA_DIR = "data/maize-crop-diagnose/data/test"
# TRAIN_DATA_DIR = "/Volumes/DOCK-HD/Data/maize-crop-diagnose/train"
# TEST_DATA_DIR = "/Volumes/DOCK-HD/Data/maize-crop-diagnose/test"

In [7]:
train_set, val_set = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DATA_DIR,
    validation_split=0.2,
    subset="both",
    seed=42,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
)
num_classes = len(train_set.class_names)
AUTOTUNE = tf.data.AUTOTUNE
train_set = train_set.prefetch(buffer_size=AUTOTUNE).cache()
val_set = val_set.prefetch(buffer_size=AUTOTUNE).cache()

Found 14749 files belonging to 3 classes.
Using 11800 files for training.
Using 2949 files for validation.
369


In [8]:
TRAIN_SIZE = 11800
TEST_SIZE = 2949

In [9]:
test_set = tf.keras.utils.image_dataset_from_directory(
    TEST_DATA_DIR,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
)
AUTOTUNE = tf.data.AUTOTUNE
test_set = test_set.prefetch(buffer_size=AUTOTUNE).cache()

Found 600 files belonging to 3 classes.


# Optimization

In [ ]:
def objective(trial):
    K.clear_session()
    train_set, val_set = tf.keras.utils.image_dataset_from_directory(
        TRAIN_DATA_DIR,
        validation_split=0.2,
        subset="both",
        seed=42,
        image_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
    )
    num_classes = len(train_set.class_names)
    AUTOTUNE = tf.data.AUTOTUNE
    train_set = train_set.prefetch(buffer_size=AUTOTUNE).cache()
    val_set = val_set.prefetch(buffer_size=AUTOTUNE).cache()

    # Architecture Hyperparameters
    size_dense = trial.suggest_int("size_dense", 128, 1024, 32)
    activation_function = trial.suggest_categorical("activation_function", ["relu", "elu"])
    # Input layers
    input = keras.layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    rescale = keras.layers.Rescaling(1.0 / 255)(input)
    conv_in = keras.layers.Conv2D(
        filters=64, kernel_size=(7, 7), strides=(2, 2), activation=activation_function
    )(rescale)
    pool_in = keras.layers.MaxPooling2D(
        pool_size=(3, 3), strides=(2, 2), padding="same"
    )(conv_in)
    # R1
    conv_r1_1 = keras.layers.Conv2D(64, 3, 1, padding="same")(pool_in)
    bn_r1_1 = keras.layers.BatchNormalization()(conv_r1_1)
    if activation_function == "relu":
        relu_r1_1 = keras.layers.ReLU()(bn_r1_1)
    else:
        relu_r1_1 = keras.layers.ELU()(bn_r1_1)
    conv_r1_2 = keras.layers.Conv2D(64, 3, 1, padding="same")(relu_r1_1)
    bn_r1_2 = keras.layers.BatchNormalization()(conv_r1_2)
    skip_r1 = keras.layers.Add()([bn_r1_2, pool_in])
    if activation_function == "relu":
        relu_r1_2 = keras.layers.ReLU()(skip_r1)
    else:
        relu_r1_2 = keras.layers.ELU()(skip_r1)
    # R2
    conv_r2_1 = keras.layers.Conv2D(64, 3, 1, padding="same")(relu_r1_2)
    bn_r2_1 = keras.layers.BatchNormalization()(conv_r2_1)
    if activation_function == "relu":
        relu_r2_1 = keras.layers.ReLU()(bn_r2_1)
    else:
        relu_r2_1 = keras.layers.ELU()(bn_r2_1)
    conv_r2_2 = keras.layers.Conv2D(64, 3, 1, padding="same")(relu_r2_1)
    bn_r2_2 = keras.layers.BatchNormalization()(conv_r2_2)
    skip_r2 = keras.layers.Add()([bn_r2_2, relu_r1_2])
    if activation_function == "relu":
        relu_r2_2 = keras.layers.ReLU()(skip_r2)
    else:
        relu_r2_2 = keras.layers.ELU()(skip_r2)
    # R3
    conv_r3_skip = keras.layers.Conv2D(128, 1, 2, padding="same")(relu_r2_2)
    conv_r3_1 = keras.layers.Conv2D(128, 3, 2, padding="same")(relu_r2_2)
    bn_r3_1 = keras.layers.BatchNormalization()(conv_r3_1)
    if activation_function == "relu":
        relu_r3_1 = keras.layers.ReLU()(bn_r3_1)
    else:
        relu_r3_1 = keras.layers.ELU()(bn_r3_1)
    conv_r3_2 = keras.layers.Conv2D(128, 3, 1, padding="same")(relu_r3_1)
    bn_r3_2 = keras.layers.BatchNormalization()(conv_r3_2)
    skip_r3 = keras.layers.Add()([bn_r3_2, conv_r3_skip])
    if activation_function == "relu":
        relu_r3_2 = keras.layers.ReLU()(skip_r3)
    else:
        relu_r3_2 = keras.layers.ELU()(skip_r3)
    # Output layers
    pool_out = keras.layers.GlobalAveragePooling2D()(relu_r3_2)
    dense_out = keras.layers.Dense(size_dense, activation=activation_function)(pool_out)
    output = keras.layers.Dense(num_classes, activation="softmax")(dense_out)
    # Model
    model = keras.Model(inputs=input, outputs=output)
    # Fitting model
    
    max_lr = trial.suggest_float("max_lr", 0.01, 0.2)
    start_lr_prop = trial.suggest_float("start_lr", 1./50, 1./5)
    last_lr = trial.suggest_float("last_lr", 0.0001, 0.01)
    onecycle = OneCycleSchedulerNoMom(
        TRAIN_SIZE // BATCH_SIZE * EPOCH,
        max_lr=max_lr,
        start_lr=start_lr_prop * max_lr,
        last_lr=last_lr,
    )
    early_stopping = tf.keras.callbacks.EarlyStopping(
        patience=10, restore_best_weights=True, start_from_epoch=20
    )

    model.compile(
        optimizer=tf.keras.optimizers.legacy.SGD(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=["accuracy"],
    )

    model.fit(
        train_set,
        validation_data=val_set,
        epochs=EPOCH,
        batch_size=BATCH_SIZE,
        callbacks=[early_stopping, onecycle],
    )
    # Evaluating and returning F1 score
    loss, acc = model.evaluate(val_set)
    return acc


study = optuna.create_study(
    storage="sqlite:///drive/MyDrive/maize-crop-diagnose/db_maize_models.sqlite3",
    study_name="resnet_onecycle_sgd",
    direction="maximize",
    load_if_exists=True,
)

study.optimize(objective, n_trials=100)
print(f"Best value: {study.best_value} (params: {study.best_params})")